In [ ]:
import numpy as np 
import pandas as pd 
import h5py
import matplotlib.pyplot as plt
import sys

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_path = '/content/gdrive/My Drive/Fooddetective/train_set_64_64_3.h5'

In [ ]:
data_set = h5py.File(data_path, 'r')

In [ ]:
data_set

<HDF5 file "train_set_64_64_3.h5" (mode r)>

In [ ]:
X_train = data_set['images'][1:1000]
X_val = data_set['images'][1000:1300]
X_test = data_set['images'][1300:1600]

In [ ]:
y_train = data_set['category'][1:1000]
y_val = data_set['category'][1000:1300]
y_test = data_set['category'][1300:1600]

In [ ]:
def set_nontrainable_layers(model):
  model.trainable = False    
  return model

In [ ]:
import tensorflow as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import Sequential
from tensorflow.keras.applications.vgg16 import VGG16

def build_model():

  vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(64, 64, 3))
  vgg_model = set_nontrainable_layers(vgg_model)
  base_model = vgg_model
  flattening_layer = layers.Flatten()
  dense_layer = layers.Dense(64, activation='relu')
  # drop_layer = layers.Dropout(0.2)
  prediction_layer = layers.Dense(101, activation='softmax')

  model = Sequential([base_model, flattening_layer, dense_layer, prediction_layer])
  return model

model = build_model()

def compile_model(model):
  model.compile(optimizer = Adam(learning_rate=0.001), 
                  metrics = 'accuracy', 
                  loss='categorical_crossentropy')
  return model


In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=(0, 1.5),
    zoom_range=(0.8, 1.2)) 

datagen.fit(X_train)

In [ ]:
y_train = y_train.astype(int)
y_val = y_val.astype(int)
y_test = y_test.astype(int)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience = 5, restore_best_weights=True)

train_flow = datagen.flow(X_train, y_train, batch_size=32)
val_flow = datagen.flow(X_val, y_val, batch_size=32)

model = build_model()
model = compile_model(model)

history = model.fit(train_flow, 
          epochs=300, 
          validation_data=(X_val, y_val), verbose=1, batch_size=32)

Epoch 1/300
32/32 [==============================] - 3s 64ms/step - loss: 13.0034 - accuracy: 0.0127 - val_loss: 4.7329 - val_accuracy: 0.0200
Epoch 2/300
32/32 [==============================] - 2s 52ms/step - loss: 4.7213 - accuracy: 0.0208 - val_loss: 4.6483 - val_accuracy: 0.0167
Epoch 3/300
32/32 [==============================] - 2s 55ms/step - loss: 4.6128 - accuracy: 0.0291 - val_loss: 4.6420 - val_accuracy: 0.0167
Epoch 4/300
32/32 [==============================] - 2s 53ms/step - loss: 4.6086 - accuracy: 0.0078 - val_loss: 4.6337 - val_accuracy: 0.0233
Epoch 5/300
32/32 [==============================] - 2s 52ms/step - loss: 4.6120 - accuracy: 0.0197 - val_loss: 4.6171 - val_accuracy: 0.0133
Epoch 6/300
32/32 [==============================] - 2s 53ms/step - loss: 4.6190 - accuracy: 0.0256 - val_loss: 4.6314 - val_accuracy: 0.0167
Epoch 7/300
32/32 [==============================] - 2s 52ms/step - loss: 4.5655 - accuracy: 0.0234 - val_loss: 4.6299 - val_accuracy: 0.0233
Epoch

In [ ]:
model.evaluate(X_test, y_test)